In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
data = pd.read_csv("Food Ingredients and Recipe Dataset with Image Name Mapping.csv")

In [ ]:
data.head()

In [ ]:
data.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
data.head()

In [ ]:
data["Title"][0]

In [ ]:
data["Ingredients"][0]

In [ ]:
data["Cleaned_Ingredients"][0]

In [ ]:
len(data)

In [ ]:
type(data["Ingredients"][0])

In [ ]:
pip install nltk

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
nltk.download('punkt_tab')

In [ ]:
import re
from nltk.tokenize import word_tokenize

def clean_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove special characters, numbers, specific units (excluding commas)
    text = re.sub(r'[^a-zA-Z\s,]', '', text)
    text = re.sub(r'\b(?:lb|tsp|tbsp|total|cup|cups|oz|teaspoons|teaspoon)\b', '', text)
    
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Join the words back into a cleaned string
    cleaned_text = ' '.join(words)
    
    return cleaned_text

# Input data as a single string
input_data = data["Ingredients"][0]

# Clean the data
cleaned_data = clean_text(input_data)

# Print the cleaned data
print(cleaned_data)

In [ ]:
data['Cleaned_Ingredients_New'] = data['Ingredients'].apply(clean_text)

In [ ]:
data["Cleaned_Ingredients_New"][3]

In [ ]:
data["Cleaned_Ingredients"][3]

In [ ]:
data.drop("Ingredients", axis=1, inplace=True)

In [ ]:
data["Cleaned_Ingredients_New"][12]

In [ ]:
data.head()

In [ ]:
#stemming


In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [ ]:
data["Cleaned_Ingredients_New"] = data["Cleaned_Ingredients_New"].apply(stem)
data

In [ ]:
#Lemmetization

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
lm = WordNetLemmatizer()

def lemm(text):
    y = []
    
    for i in text.split():
        y.append(lm.lemmatize(i))
        
    return " ".join(y)

In [ ]:
data["Cleaned_Ingredients_New"] = data["Cleaned_Ingredients_New"].apply(lemm)

In [ ]:
data["Cleaned_Ingredients_New"][0]

In [ ]:
#Text Vectorization

In [ ]:
# Text Vectorization using Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(data["Cleaned_Ingredients_New"]).toarray()

In [ ]:
vectors.shape

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
# Cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
data["Title"][2]

In [ ]:
data[data['Title'] == 'Miso-Butter Roast Chicken With Acorn Squash Panzanella'].index[0]

In [ ]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

In [ ]:
def recommend(recipe):
    recipe_index = data[data['Title'] == recipe].index[0]
    distances = similarity[recipe_index]
    recipe_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in recipe_list:
        print(data["Title"][i[0]])

In [ ]:
recommend('Miso-Butter Roast Chicken With Acorn Squash Panzanella')

In [ ]:
#using count vectorizer

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# # Load your dataset
# data = pd.read_csv("Food Ingredients and Recipe Dataset with Image Name Mapping.csv")

# # Drop the 'Unnamed: 0' column
# data.drop('Unnamed: 0', inplace=True, axis=1)

# # Clean the text data
# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r'[^a-zA-Z\s,]', '', text)
#     text = re.sub(r'\b(?:lb|tsp|tbsp|total|cup|cups)\b', '', text)
#     words = word_tokenize(text)
#     cleaned_text = ' '.join(words)
#     return cleaned_text

# def stem(text):
#     y = []
#     for i in text.split():
#         y.append(ps.stem(i))
#     return " ".join(y)

# ps = PorterStemmer()
# data["Cleaned_Ingredients_New"] = data["Ingredients"].apply(clean_text)
# data["Cleaned_Ingredients_New"] = data["Cleaned_Ingredients_New"].apply(stem)

# # Create a CountVectorizer for ingredient vectors
cv = CountVectorizer(max_features=5000, stop_words='english')
ingredient_vectors = cv.fit_transform(data["Cleaned_Ingredients_New"]).toarray()

In [ ]:
# Calculate cosine similarity between input ingredients and dataset ingredients
def recommend_dishes(input_ingredients, dataset, vectors, vectorizer, top_n=5):
    # Clean and stem the input ingredients
    cleaned_input = clean_text(input_ingredients)
    stemmed_input = stem(cleaned_input)
    
    # Transform the input ingredients into a vector
    input_vector = vectorizer.transform([stemmed_input]).toarray()
    
    # Calculate cosine similarity
    similarities = cosine_similarity(input_vector, vectors)
    
    # Get the indices of the top N similar dishes
    top_indices = np.argsort(similarities[0])[::-1][:top_n]
    
    # Get the titles and instructions of the top N similar dishes
    top_dishes = dataset.iloc[top_indices][["Title", "Instructions"]]
    
    return top_dishes

In [ ]:
# Input your ingredients as a string
input_ingredients = "whole chicken, kosher salt, freshli ground pepper, unsalt butter, extravirgin olive oil, small red onion, apple"

# Get the top 5 recommended dishes
recommended_dishes = recommend_dishes(input_ingredients, data, ingredient_vectors, cv)

# Print the recommended dishes and their instructions
print("Top 5 Recommended Dishes:")
for i, (dish, instructions) in enumerate(recommended_dishes.iterrows(), start=1):
    print(f"{i}. {instructions['Title']}")
    print("   Instructions:", instructions['Instructions'])
    print()

In [ ]:
from PIL import Image
import IPython.display as display

In [ ]:
data.head()

In [ ]:
# Print the recommended dishes with instructions, image names, and original dataset index
print("Top 5 Recommended Dishes:")
for i, (index, row) in enumerate(recommended_dishes.iterrows(), start=1):
    title = row['Title']
    instructions = row['Instructions']
    image_name = data['Image_Name'][index]  # Use the correct column name 'Image_Name'
    original_index = index  # Get the original dataset index
    ingredients = data['Cleaned_Ingredients'][index]  # Get the ingredients from 'Cleaned_Ingredients'
    
    # Load and display the image
    image_path = f"pexels-lum3n-44775-1410236.jpg"  # Update the path to your image folder
    img = Image.open(image_path)
    display.display(img)
    
    print("\n")
    print(f"Title: {title}")
    print("\n")
    print(f"Ingredients: \n{ingredients}")
    print("\n")
    print(f"Instructions: \n{instructions}\n")
    print("\n")
    print("\n")

In [ ]:
import pickle
pickle.dump(data.to_dict(), open('Recipie.pkl', 'wb'))

In [ ]:
pickle.dump(similarity, open('RecipieSimilarity.pkl', 'wb'))

In [ ]:
pickle.dump(vectorizer, 'vectorizer.pkl')

In [ ]:
pickle.dump(Cleaned_Ingredients_New,'dataset.pkl')